Matt Dula

Collaborators: Cory Hilton, Anton Schlegel, Derek Bowman

# Memory and Performance

The lecture covered how data flows through a computer in order to execute our code. Most importantly, we introduced the memory hierarchy, the fetch-execute-store cycle, and the basic idea of how data is laid out in memory (also called **data locality**)

In this workbook, you'll put these ideas together to learn:
* why NumPy arrays are faster than lists for particular kinds of applications
* why some methods of creating lists are faster than others

Future lessons will build on this understanding of data locality and how it impacts performance, as well as the ability to time code execution.

---
## Diagraming Memory Flow

In the last part of lecture, we introduced **cache lines** as well as the implementation underlying **Python lists** and **NumPy arrays**. 

The structure of NumPy arrays let's them take advantage of cache lines due to high data locality.

For simplicity in the following exercises, **you only need to consider RAM and L1 cache.** We'll use these to represent the whole memory hierarchy (e.g. ingore L2 and L3 cahces). Assume the L1 cache is large enough that it can hold many cache lines and we don't have to worry about data being deleted from it. Also assume that the CPU's registers are so small it is very unlikely that they see any data re-use.

We will also assume that **a cache line can fit 8 floating point numbers.**

### NumPy Arrays

Imagine we have created an array with the following command:

```python
a = np.ones(100)
```

This array has size 100, defaults to holding floating point numbers, and every value is set to `1.0`. Remember that arrays store their data in a block of contiguous memory address. It might be helpful to image the array like table in memory:

| `a[0]` | `a[1]` | `a[2]` | `a[3]` | ... |
|--------|--------|--------|--------|-----|
| `1.0`  | `1.0`  | `1.0`  | `1.0`  | ... |


Some time after we created the array, we need to access it for calculations. At this time, the array is only present in RAM. We loop through the array as follows:

```python
for i in range(a.size):
    a[i] *= 5
```


---
#### Exercises

1. Which entry in array `a` does the CPU request first? Where is it **first** able to find this value, RAM or L1 cache?

The CPU requests `a[0]` first. It finds this value in RAM first because it is only present there (described above).

2. After the CPU requests `a[0]`, what's the highest index of `a` available in L1 cache?

The highest index available in the cache line is `i=7`, as the cache line is assumed to fit 8 floating point numbers.

3. When `i=3`, where is the CPU **first** able to find this value? (i.e., on what level does it stop looking through the memory hierarchy)

It finds this value in the L1 cache because 3 is less than 7, so this index has been retrieved as part of the previous cache line.

4. Later, when `i=8`, where is the CPU **first** able to find this value? What action does this trigger?

Since `a[8]` is outside the previous cache line, it finds this value in RAM. This triggers a new cache line fetch.

---
### Python Lists

Now imagine we have created a list that is similar to `a`; that is, it hold 100 floating point numbers, all of which are `1.0`. We'll do so with a [list comprehension](https://www.w3schools.com/python/python_lists_comprehension.asp). We'll explore this technique for creating lists later in this workbook.

```python
l = [1.0 for i in range(100)]
```

Like arrays, lists are also stored as contiguous blocks of memory addresses; *however* each of these blocks is a *pointer*. The actual data we are using a list to store is kept at some other memory address that is referenced by the pointer.

Thinking in terms of a table, the list looks like this:

| `l[0]` | `l[1]` | `l[2]` | `l[3]` | ... |
|--------|--------|--------|--------|-----|
| `0x3e8` | `0x3e9` | `0x578` | `0x7d0`  | ... |

Here, we're using **hexadecimal numbers** (i.e., numbers written in base 16) to refer to memory addresses. These numbers start with `0x` and use the letters `a` through `f` to represent digits 10 through 15. They are commonly used to represent memory addresses.

The pointers above may or may not point to contiguous blocks of memory. For instance, `l[0]` and `l[1]` both reference coniguous addresses:

| `0x3e8` | `0x3e9` |
|---------|---------|
| `1.0`   | `1.0`   |

Other entries, such as `l[2]`, are not contiguous with any other data in the list.

In order to access the data associated with `l[0]`, we must first fetch the pointer associated with `l[0]`, and then the data stored at the address indicated by the pointer.

Some time after we created the list, we need to access it for calculations. At this time, the list is only present in RAM. We loop through the list as follows:

```python
for i in range(a.size):
    l[i] *= 5
```

Be aware that cache lines are used **any** time we need to move data. The computer is *hoping* that data in the cache line will be used soon in the future, but this does not have to be true. Remember that for this example, we can assume the L1 cache holds several cache lines.

---
#### Exercises

1. When the CPU tries to access the data associated with `l[0]`, how many cache lines are needed to transfer this data out of RAM? Why are these cache lines needed? **warning** some students interpreted this as "cache lines overall" in which case the answer is always 2. Clarify it's about what new cache line fetches it triggers.

2 cache line fetches are needed: one to request the list values (pointers) and one to request the values starting at the memory location referenced by `l[0]`.

2. When the CPU tries to access the data associated with `l[1]`, how many cache lines are needed to transfer this data out of RAM? What data (if any) is already present in the L1 cache?

No cache lines are fetched, because the pointer associated with `l[1]` has been loaded as part of the `l[0]` cache line fetch, and since the memory address of `l[1]` happens to be within the cache line of the memory address of `l[0]`, we can access the value without a cache line fetch.

3. Next, when the CPU tries to access the data associated with `l[2]`, how many cache lines are needed to transfer this data out of RAM? What data (if any) is already present in the L1 cache?

We need one cache line fetch. The pointer value of `l[2]` has already been fetched, but that actual block of memory has not been fetched.

---
## Building Lists Efficiently

In the pre-class reading, you learned about how `append` operations for lists are implemented under the hood. Memory allocation is generally very slow, so repeatedly appending to a list is generally not optimal. It's best to create a list with all desired elements already present. But how to do that without writing out every element by hand?

Enter **list comprehensions**, a feature of Python that allows you to assemble lists (and dictionaries!) efficiently. The basic syntax of a comprehension is as follows:

```python
[ <operation> for <var> in <iterable object>]
```

Notice that the syntax mirrors a typical `for` loop, but stuffed inside square brackets (a dictionary comprehension would use curly braces). Any modification of data that we would typically put inside a `for` loop goes at the front of the comprehension.

For example, the loop
```python
lst = []
for i in range(50):
    lst.append(i**2)
```
becomes
```python
lst = [i**2 for i in range(50)]
```

---
### Exercises


1. Time the following block of code to establish a baseline to compare performance improvements against.
```python
lst = []
shift = 5
for i in range(1000):
    a = i + shift
    lst.append( a**2/2 )
```

In [3]:
%%timeit
lst = []
shift = 5
for i in range(1000):
    a = i + shift
    lst.append( a**2/2 )

137 μs ± 436 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


2. Rewrite the above code to use a list comprehension. Don't forget to check the accuracy of your result.

In [8]:
%%timeit
lst2 = [(i+5)**2/2 for i in range(1000)]
# checked for accuracy using print, works correctly

122 μs ± 740 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


3. Time the list comprehension you wrote above. How many times faster is this version of the code?

My version is 1.12 times faster (122us vs. 137us)

4. You can include logic statements in a list (or dictionary) comprehension.
For example, we can add an `if`-`else` statement as follows:
`[ <operation> if <condition> else <alternative operation> for <var> in <iterable object> ]`. Use this approach to convert the following code into a list comprehension. Make sure to check the results.

```python
words = ["bats","dog","bowls","cars","house"]
singles = []
for w in words:
    if w[-1] == 's':
        singles.append(w[:-1]) # exclude final 's'
    else:
        singles.append(w)
```

In [10]:
words = ["bats","dog","bowls","cars","house"]
singles = [w[:-1] if w[-1] == 's' else w for w in words]
print(singles)

['bat', 'dog', 'bowl', 'car', 'house']


5. Can you think of a situation in which you *wouldn't* be able to use a list comprehension? In such a case, you may have to resort to the repeated `append` method.

If the creation of the list requires reference to a previous value in the list, comprehension would not work (or at least would be very complicated) because there is no way to self-reference the array that has not been created yet.

---
## Vectorization 

Sometimes, a CPU can apply the same instruction to multiple pieces of data at the same time in a processes called **vectorization.** Consider our NumPy example from before:

```python
a = np.ones(100)
for i in range(a.size):
    a[i] * 5
```

In its current form, the CPU can take advantage of cache lines, but it *cannot* employ vectorization. Simply put, we are relying too much on "pure" Python for vectorization to take place. Python's way of handling instructions does not make it clear to the CPU that vectorization is possible.

That said, *NumPy* supports vectorization due to how its implemented (in C).
We can rewrite our `for` loop to take advantage of this:

```python
a = np.ones(100)
a * 5
```

---
### Exercises

1. Time the `for` loop displayed above. Don't include array creation or module import in your timing measurement!

In [7]:
import numpy as np

In [14]:
%%timeit
a = np.ones(100)
for i in range(a.size):
    a[i] * 5

20.4 μs ± 361 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


2. Time the vectorized NumPy statement displayed above. How many times faster is this version?

In [15]:
%%timeit
a = np.ones(100)
a * 5

6.78 μs ± 49.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


The vectorized version is 3 times faster (6.7us vs. 20.4us)

3. Time the following purely "Pythonic" code to establish a baseline to compare performance improvements against. Exclude the import statement from your timing.

```python
import math

b = []
for i in range(500):
    b.append(i * math.sin(i))
```

In [16]:
import math

In [17]:
%%timeit
b = []
for i in range(500):
    b.append(i * math.sin(i))

68.2 μs ± 79.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


4. Rewrite the above code to be more "NumPythonic" in order to take advantage of vectorization. The NumPy [array creation](https://numpy.org/doc/1.25/reference/routines.array-creation.html#numerical-ranges) and [math](https://numpy.org/doc/1.25/reference/routines.math.html) documentation pages are good references. Make sure to test the results of your refactor!

In [19]:
%%timeit
i = np.arange(500)
b = i * np.sin(i)

18.1 μs ± 66.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


5. Time the result of your refactor. What factor of speedup did you achieve? (i.e., what is the ratio between the old time and the new time?)

The result of the refactor is 3.8 times faster (18.1us vs. 68.2us).

6. Using what you've learned about caches and cache lines, explain why vectorization is possible and why it lets code run faster.

Vectorization is possible because lists require at most two cache line fetches per reference to a value, while NumPy array references require at most one cache line reference. In short, NumPy caches the values directly, instead of the addresses.

---
## A Final Comparison

In this workbook, we've explored different ways to work with lists and NumPy arrays. We've also learned details about how these objects work under the hood, and how their implementations can bring benefits and drawbacks.

Now, we will directly compare these methods against each other. You will implement a vector norm (also called the "dot product") in several different ways:

1. Looping over a list
2. Using a list comprehension and the built-in `sum` function
3. Looping over a NumPy array
4. Using `np.sum()`
5. Using `np.dot()`

For example, for a vector with 1 million elements, item #4 would be implemented as
```python
v = np.arange(1e6)
norm = np.sum(v * v)
```

Then, **time how long it takes to calculate `norm` with each implementation.** Make sure you test the same vector each time.

You are encouraged to look up the documentation for any functions you are unfamiliar with.

Below are the implementations:

In [13]:
%%timeit
vec = []
norm = 0
for i in range(1000000):
    vec.append(i)

for v in vec:
    norm += v * v

116 ms ± 1.01 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [15]:
%%timeit
norm = sum([v * v for v in range(1000000)])

79.3 ms ± 270 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [17]:
%%timeit
vec = np.arange(1e6)
norm = 0
for v in vec:
    norm += v * v

167 ms ± 6.74 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [18]:
%%timeit
v = np.arange(1e6)
norm = np.sum(v * v)

2.19 ms ± 23.6 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [19]:
%%timeit
v = np.arange(1e6)
norm = np.dot(v, v)

1.32 ms ± 18.9 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


2. Using what you've learned this lesson, explain why you think the ranking turned out the way it is. What, if anything, can you not explain?

The list-based methods clearly take longer, because they again require more cache list fetches. There is a speedup using the `sum` function, likely because it makes use of values remaining in the cache and register and summing them all at once as opposed to alternating between value creation and summing. I cannot explain why the NumPy iteration takes longer than the list iteration. However, the NumPy vectorized multiplication inside of the `np.sum` function speeds up the process significantly by taking advantage of the cached values during multiplication and then again the easily-accessible cached values of the NumPy array during summation. Finally, the `np.dot` function uses a similar process, using cached summands and products together.

3. What have you taken away from this comparison?

I have a two key takeaways: first, use NumPy as much as possible for numerical solutions to problems, as the caching of values instead of address offers significant speedups. Second, try to perform as many operations as possible on the already-cached values before requiring the next cache list fetch, as this will speed up the process once again.

---
## For Next Class

* Make note of items you'd like to discuss further in the next class.
* **If you're new to Pandas,** I suggest skimming the [Pandas tutorials](https://pandas.pydata.org/docs/getting_started/intro_tutorials/).